# Fabric User Activity Monitoring

Collects **user activity logs** and **access patterns** from Fabric REST APIs and sends to Azure Log Analytics.

## ⚠️ **Admin Permissions Required**

This notebook uses the `/admin/activities` endpoint which requires:
- **Fabric Administrator** role in your tenant
- **Service Principal** must be granted admin consent
- **Tenant-level permissions** for security monitoring

## 🛡️ **Enterprise Security Monitoring**

Perfect for:
- **Security teams** monitoring user access patterns  
- **Compliance auditing** across all workspaces
- **Suspicious activity detection** tenant-wide
- **Data governance** and access tracking

In [ ]:
# === One-time installs per session Or Use Fabric Environment ===
%pip install --quiet msal requests azure-identity azure-keyvault-secrets python-dotenv

In [39]:
# === Parameters (mark this as a parameter cell in Fabric) ===
import os
from dotenv import load_dotenv
load_dotenv()

# Multi-workspace monitoring for security and compliance
# Empty list = monitor ALL accessible workspaces (recommended for security)
# Specific workspaces = ["workspace-id-1", "workspace-id-2"]
workspace_ids = []

# Note: Admin Activities API typically supports 1-7 days max
lookback_hours = 720

dcr_endpoint_host = os.getenv("DCR_ENDPOINT_HOST")
dcr_immutable_id = os.getenv("DCR_IMMUTABLE_ID")
stream_user_activity = "Custom-FabricUserActivity_CL"

tenant_id = os.getenv("FABRIC_TENANT_ID")
client_id = os.getenv("FABRIC_APP_ID")
client_secret_env = os.getenv("FABRIC_APP_SECRET")

use_key_vault = False
use_managed_identity = False
key_vault_uri = os.getenv("AZURE_KEY_VAULT_URI", "https://kaydemokeyvault.vault.azure.net/")
key_vault_secret_name = os.getenv("AZURE_KEY_VAULT_SECRET_NAME", "FabricServicePrincipal")

if not all([tenant_id, client_id, dcr_endpoint_host, dcr_immutable_id]):
    missing = []
    if not tenant_id: missing.append("FABRIC_TENANT_ID")
    if not client_id: missing.append("FABRIC_APP_ID")
    if not dcr_endpoint_host: missing.append("DCR_ENDPOINT_HOST")
    if not dcr_immutable_id: missing.append("DCR_IMMUTABLE_ID")
    print(f"❌ Missing: {', '.join(missing)}")
else:
    print("✅ Environment variables loaded")

if workspace_ids:
    print(f"Monitoring {len(workspace_ids)} specific workspaces")
else:
    print("Monitoring ALL accessible workspaces (security mode)")
print(f"Lookback: {lookback_hours} hours")

✅ Environment variables loaded
Monitoring ALL accessible workspaces (security mode)
Lookback: 720 hours


In [40]:
# === define main functions ===
import os, json, time, datetime as dt
import requests
from typing import List, Dict, Any

def get_secret_from_kv(vault_uri: str, secret_name: str, tenant_id: str = None, client_id: str = None, client_secret: str = None, use_managed_identity: bool = False) -> str:
    try:
        from azure.keyvault.secrets import SecretClient
        if use_managed_identity:
            from azure.identity import ManagedIdentityCredential
            credential = ManagedIdentityCredential()
        else:
            from azure.identity import ClientSecretCredential
            credential = ClientSecretCredential(tenant_id=tenant_id, client_id=client_id, client_secret=client_secret)
        client = SecretClient(vault_url=vault_uri, credential=credential)
        return client.get_secret(secret_name).value
    except Exception as e:
        print(f"[KeyVault] Failed: {e}")
        return None

FABRIC_SCOPE = "https://api.fabric.microsoft.com/.default"
MONITOR_SCOPE = "https://monitor.azure.com/.default"
FABRIC_API = "https://api.fabric.microsoft.com/v1"

def acquire_token_client_credentials(tenant: str, client_id: str, client_secret: str, scope: str) -> str:
    import msal
    authority = f"https://login.microsoftonline.com/{tenant}"
    app = msal.ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)
    result = app.acquire_token_for_client(scopes=[scope])
    if "access_token" not in result:
        raise RuntimeError(f"Failed to get token for {scope}: {result}")
    token = result["access_token"]
    print(f"✅ Token acquired for {scope}")
    return token

def iso_now() -> str:
    return dt.datetime.utcnow().replace(tzinfo=dt.timezone.utc).isoformat().replace("+00:00", "Z")

def parse_iso(s: str) -> dt.datetime:
    if not s:
        return None
    if s.endswith("Z"):
        s = s[:-1] + "+00:00"
    parsed = dt.datetime.fromisoformat(s)
    if parsed.tzinfo is None:
        parsed = parsed.replace(tzinfo=dt.timezone.utc)
    return parsed

def get_workspaces(token: str) -> List[Dict[str, Any]]:
    """Get accessible workspaces for fallback monitoring"""
    url = f"{FABRIC_API}/workspaces"
    headers = {"Authorization": f"Bearer {token}"}
    
    try:
        r = requests.get(url, headers=headers, timeout=60)
        r.raise_for_status()
        return r.json().get("value", [])
    except Exception as e:
        print(f"❌ Failed to get workspaces: {e}")
        return []

def get_user_activities(workspace_ids: List[str], token: str, start_time: dt.datetime, end_time: dt.datetime) -> List[Dict[str, Any]]:
    # Primary endpoint - Admin Activities (requires admin permissions)
    url = f"{FABRIC_API}/admin/activities"
    headers = {"Authorization": f"Bearer {token}"}
    
    start_str = start_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
    end_str = end_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
    
    params = {
        "startDateTime": start_str,
        "endDateTime": end_str,
    }
    
    activities = []
    continuation_token = None
    
    print(f"🔍 Trying admin activities endpoint...")
    
    while True:
        if continuation_token:
            params["continuationToken"] = continuation_token
        
        try:
            r = requests.get(url, headers=headers, params=params, timeout=60)
            
            if r.status_code == 404:
                print("❌ Admin activities endpoint not accessible (404)")
                print("💡 This endpoint requires Fabric Administrator permissions")
                print("💡 Falling back to workspace-level monitoring...")
                return get_workspace_activities_fallback(workspace_ids, token, start_time, end_time)
            elif r.status_code == 403:
                print("❌ Access denied to admin activities endpoint (403)")
                print("💡 Service principal needs Fabric Administrator role")
                print("💡 Falling back to workspace-level monitoring...")
                return get_workspace_activities_fallback(workspace_ids, token, start_time, end_time)
            
            r.raise_for_status()
            data = r.json()
            
            batch_activities = data.get("activityEventEntities", [])
            
            # Filter by workspace_ids if specified (empty list = all workspaces)
            if workspace_ids:
                batch_activities = [a for a in batch_activities if a.get("WorkspaceId") in workspace_ids]
            
            activities.extend(batch_activities)
            continuation_token = data.get("continuationToken")
            
            if not continuation_token:
                break
                
        except requests.exceptions.RequestException as e:
            print(f"❌ Request failed: {e}")
            if hasattr(e, 'response') and e.response:
                print(f"Status: {e.response.status_code}")
                print(f"Response: {e.response.text[:500]}")
            break
        except Exception as e:
            print(f"❌ Unexpected error: {e}")
            break
    
    return activities

def get_workspace_activities_fallback(workspace_ids: List[str], token: str, start_time: dt.datetime, end_time: dt.datetime) -> List[Dict[str, Any]]:
    """Fallback: Get activities from individual workspaces when admin API unavailable"""
    print("🔄 Using workspace-level activity monitoring...")
    
    # Get workspaces to monitor
    if not workspace_ids:
        workspaces = get_workspaces(token)
        workspace_ids = [w["id"] for w in workspaces[:10]]  # Limit to 10 for demo
        print(f"Found {len(workspaces)} workspaces, monitoring first 10")
    
    activities = []
    headers = {"Authorization": f"Bearer {token}"}
    
    for workspace_id in workspace_ids:
        try:
            # Get workspace information
            workspace_url = f"{FABRIC_API}/workspaces/{workspace_id}"
            workspace_resp = requests.get(workspace_url, headers=headers, timeout=30)
            
            if workspace_resp.status_code == 200:
                workspace_info = workspace_resp.json()
                
                # Create synthetic activity record for workspace access
                activity = {
                    "Id": f"workspace-access-{workspace_id}-{int(time.time())}",
                    "RecordType": "WorkspaceAccess",
                    "CreationTime": iso_now(),
                    "Operation": "WorkspaceViewed",
                    "WorkspaceId": workspace_id,
                    "WorkspaceName": workspace_info.get("displayName", "Unknown"),
                    "Activity": "WorkspaceAccess",
                    "ActivityType": "WorkspaceViewed",
                    "UserId": "ServicePrincipal",
                    "UserType": "ServicePrincipal",
                    "Workload": "PowerBI"
                }
                activities.append(activity)
                
        except Exception as e:
            print(f"⚠️  Workspace {workspace_id}: {e}")
            continue
    
    print(f"✅ Collected {len(activities)} workspace access activities")
    return activities

def map_user_activity(activity: Dict[str, Any]) -> Dict[str, Any]:
    return {
        "TimeGenerated": activity.get("CreationTime") or iso_now(),
        "ActivityId": activity.get("Id"),
        "RecordType": activity.get("RecordType"),
        "CreationTime": activity.get("CreationTime"),
        "Operation": activity.get("Operation"),
        "OrganizationId": activity.get("OrganizationId"),
        "UserType": activity.get("UserType"),
        "UserKey": activity.get("UserKey"),
        "Workload": activity.get("Workload"),
        "UserId": activity.get("UserId"),
        "ClientIP": activity.get("ClientIP"),
        "UserAgent": activity.get("UserAgent"),
        "Activity": activity.get("Activity"),
        "ActivityType": activity.get("ActivityType"),
        "ItemName": activity.get("ItemName"),
        "WorkspaceId": activity.get("WorkspaceId"),
        "WorkspaceName": activity.get("WorkspaceName"),
        "CapacityId": activity.get("CapacityId"),
        "CapacityName": activity.get("CapacityName"),
        "AppName": activity.get("AppName"),
        "ObjectId": activity.get("ObjectId"),
        "DatasetName": activity.get("DatasetName"),
        "ReportName": activity.get("ReportName"),
        "ReportType": activity.get("ReportType"),
        "RequestId": activity.get("RequestId"),
        "AppReportId": activity.get("AppReportId"),
        "DistributionMethod": activity.get("DistributionMethod"),
        "ConsumptionMethod": activity.get("ConsumptionMethod"),
    }

def post_rows_to_dcr(endpoint_host: str, dcr_id: str, stream_name: str, rows: List[Dict[str, Any]], monitor_token: str):
    if not rows:
        return {"sent": 0, "batches": 0}
    
    MAX_BYTES = 950_000
    batch, batches, sent, size = [], 0, 0, 2
    
    def flush():
        nonlocal batches, sent, batch, size
        if not batch:
            return
        url = f"https://{endpoint_host}/dataCollectionRules/{dcr_id}/streams/{stream_name}?api-version=2023-01-01"
        headers = {"Authorization": f"Bearer {monitor_token}", "Content-Type": "application/json"}
        resp = requests.post(url, headers=headers, data=json.dumps(batch), timeout=60)
        if resp.status_code >= 400:
            raise RuntimeError(f"Ingestion failed ({resp.status_code}): {resp.text[:500]}")
        batches += 1
        sent += len(batch)
        batch, size = [], 2
    
    for row in rows:
        s = len(json.dumps(row, separators=(",", ":")))
        if size + s + (1 if batch else 0) > MAX_BYTES:
            flush()
        batch.append(row)
        size += s + (1 if batch else 0)
    flush()
    return {"sent": sent, "batches": batches}

print("✅ Functions loaded (with workspace-level fallback)")

✅ Functions loaded (with workspace-level fallback)


In [41]:
# === Main ===
client_secret = None

if client_secret_env:
    client_secret = client_secret_env
    print("✅ Using environment variable")
elif use_key_vault:
    if use_managed_identity:
        client_secret = get_secret_from_kv(key_vault_uri, key_vault_secret_name, use_managed_identity=True)
    else:
        temp_secret = os.getenv("FABRIC_APP_SECRET")
        if not temp_secret:
            raise RuntimeError("FABRIC_APP_SECRET required for Key Vault access")
        client_secret = get_secret_from_kv(key_vault_uri, key_vault_secret_name, tenant_id, client_id, temp_secret, use_managed_identity=False)

if not client_secret:
    raise RuntimeError("Client secret not found")

print("✅ Client secret resolved")

fabric_token = acquire_token_client_credentials(tenant_id, client_id, client_secret, FABRIC_SCOPE)
monitor_token = acquire_token_client_credentials(tenant_id, client_id, client_secret, MONITOR_SCOPE)

# Test admin access before proceeding
print("🔍 Testing admin API access...")
test_url = f"{FABRIC_API}/admin/activities"
test_params = {
    "startDateTime": "2025-09-10T00:00:00.000Z",
    "endDateTime": "2025-09-10T01:00:00.000Z"
}
headers = {"Authorization": f"Bearer {fabric_token}"}

try:
    test_response = requests.get(test_url, headers=headers, params=test_params, timeout=30)
    if test_response.status_code == 200:
        print("✅ Admin API access confirmed!")
    elif test_response.status_code == 404:
        print("❌ Admin activities endpoint not found (404)")
        print("📋 TO FIX: Contact your Fabric Administrator to:")
        print("   1. Enable 'Service principals can access Fabric APIs'")
        print("   2. Add your service principal to admin API allowed list")
        print("   3. Grant Fabric Administrator role to service principal")
        print(f"   4. Service Principal ID: {client_id}")
    elif test_response.status_code == 403:
        print("❌ Access denied to admin activities (403)")
        print("📋 TO FIX: Your service principal needs Fabric Administrator role")
        print(f"   Service Principal ID: {client_id}")
    else:
        print(f"❌ Unexpected response: {test_response.status_code}")
        print(f"Response: {test_response.text[:200]}")
except Exception as e:
    print(f"❌ Connection test failed: {e}")

now = dt.datetime.utcnow().replace(tzinfo=dt.timezone.utc)
start_time = now - dt.timedelta(hours=lookback_hours)

print(f"\nCollecting activities from {start_time.isoformat()} to {now.isoformat()}")

activities = get_user_activities(workspace_ids, fabric_token, start_time, now)
print(f"Found {len(activities)} activities")

activity_rows = []
for activity in activities:
    activity_rows.append(map_user_activity(activity))

summary = {}
if activity_rows:
    print("Sending user activities...")
    result = post_rows_to_dcr(dcr_endpoint_host, dcr_immutable_id, stream_user_activity, activity_rows, monitor_token)
    summary["user_activities"] = result
else:
    print("ℹ️  No activities to send (check admin permissions above)")

print("✅ Done!")
print(json.dumps(summary, indent=2))

✅ Using environment variable
✅ Client secret resolved
✅ Token acquired for https://api.fabric.microsoft.com/.default
✅ Token acquired for https://api.fabric.microsoft.com/.default
✅ Token acquired for https://monitor.azure.com/.default
🔍 Testing admin API access...
✅ Token acquired for https://monitor.azure.com/.default
🔍 Testing admin API access...
❌ Admin activities endpoint not found (404)
📋 TO FIX: Contact your Fabric Administrator to:
   1. Enable 'Service principals can access Fabric APIs'
   2. Add your service principal to admin API allowed list
   3. Grant Fabric Administrator role to service principal
   4. Service Principal ID: f4b66b80-24d3-4498-9cdf-02f47c776315

🔍 Trying admin activities endpoint...
❌ Admin activities endpoint not found (404)
📋 TO FIX: Contact your Fabric Administrator to:
   1. Enable 'Service principals can access Fabric APIs'
   2. Add your service principal to admin API allowed list
   3. Grant Fabric Administrator role to service principal
   4. Serv